<a href="https://colab.research.google.com/github/m0zzarella/Alzheimer-s-Lewy-Body-Dementia-Frontotemporal-Dementia-Feature-Detection/blob/main/SVM_Random_Forest_Classification_of_fMRI_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = "/content/drive/MyDrive/Alzheimer_Classification"

In [4]:
import numpy as np
import cv2
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [7]:

disease_folder = '/content/drive/MyDrive/Alzheimer_Classification/AD_Data'
control_folder = '/content/drive/MyDrive/Alzheimer_Classification/CN_Data'

disease_label = 1
control_label = 0

In [15]:
disease_images = []
disease_labels = []


disease_count = 0
normal_count = 0
for filename in tqdm(os.listdir(disease_folder)):
    img = cv2.imread(os.path.join(disease_folder, filename))
    if img is not None:
        img = cv2.resize(img, (128, 128))
        disease_images.append(img)
        disease_labels.append(disease_label)
        disease_count+=1


control_images = []
control_labels = []
for filename in tqdm(os.listdir(control_folder)):
    img = cv2.imread(os.path.join(control_folder, filename))
    if img is not None:
        img = cv2.resize(img, (128, 128))
        control_images.append(img)
        control_labels.append(control_label)
        normal_count+=1

100%|██████████| 1032/1032 [00:03<00:00, 288.87it/s]


In [16]:
images = np.concatenate((disease_images, control_images))
labels = np.concatenate((disease_labels, control_labels))


train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


train_images = train_images.reshape(train_images.shape[0], -1)
test_images = test_images.reshape(test_images.shape[0], -1)

print("Length of:")
print(f"Training Images = {len(train_images)}")
print(f"Testing Images = {len(test_images)}")
print(f"Train Labels = {len(train_labels)}")
print(f"Test Labels = {len(test_labels)}")
print(f"-No. of Class 0 (Cognitive Normal) samples = {normal_count}-")
print(f"-No. of Class 1 (Alzheimer's Disease) samples = {disease_count}-")


Length of:
Training Images = 2132
Testing Images = 534
Train Labels = 2132
Test Labels = 534
-No. of Class 0 (Cognitive Normal) samples = 1032-
-No. of Class 1 (Alzheimer's Disease) samples = 1634-


In [17]:

#SVM models
linear_svm = SVC(kernel='linear')
rbf_svm = SVC(kernel='rbf')

#train the SVM models
linear_svm.fit(train_images, train_labels)
rbf_svm.fit(train_images, train_labels)

#predict the labels for the test images
linear_preds = linear_svm.predict(test_images)
rbf_preds = rbf_svm.predict(test_images)

#calculate the accuracy, precision, recall, and F1-score scores
linear_acc = accuracy_score(test_labels, linear_preds)
rbf_acc = accuracy_score(test_labels, rbf_preds)

linear_precision = precision_score(test_labels, linear_preds)
rbf_precision = precision_score(test_labels, rbf_preds)

linear_recall = recall_score(test_labels, linear_preds)
rbf_recall = recall_score(test_labels, rbf_preds)

linear_f1 = f1_score(test_labels, linear_preds)
rbf_f1 = f1_score(test_labels, rbf_preds)


In [18]:
from sklearn.metrics import confusion_matrix



tn1, fp1, fn1, tp1 = confusion_matrix(test_labels, linear_preds).ravel()
# Calculate sensitivity and specificity
sensitivity_linear = tp1 / (tp1 + fn1)
specificity_linear = tn1 / (tn1 + fp1)

tn2, fp2, fn2, tp2 = confusion_matrix(test_labels, rbf_preds).ravel()
# Calculate sensitivity and specificity
sensitivity_rbf = tp2 / (tp2 + fn2)
specificity_rbf = tn2 / (tn2 + fp2)


print("--- Linear Kernel Metrics Performances --- ")

print(f"Specificity Linear: {specificity_linear:.2f}")
print(f"Sensitivity Linear: {sensitivity_linear:.2f}")

print(f"Linear SVM accuracy: { linear_acc: 0.2f}")
print(f"Linear SVM precision: { linear_precision: .2f}")
print(f"Linear SVM recall: {linear_recall: .2f}")
print(f"Linear SVM F1-score: {linear_f1: 0.2f}")






--- Linear Kernel Metrics Performances --- 
Specificity Linear: 0.82
Sensitivity Linear: 0.89
Linear SVM accuracy:  0.86
Linear SVM precision:  0.89
Linear SVM recall:  0.89
Linear SVM F1-score:  0.89


In [19]:
print("--- RBF Kernel Metrics Performances --- ")

print(f"Specificity_RBF: {specificity_rbf:.2f}")
print(f"Sensitivity RBF: {sensitivity_rbf:.2f}")
print(f"RBF SVM accuracy: { rbf_acc: 0.2f}")
print(f"RBF SVM precision: { rbf_precision: 0.2f}")
print(f"RBF SVM recall: { rbf_recall: 0.2f}")
print(f"RBF SVM F1-score: { rbf_f1: 0.2f}")

--- RBF Kernel Metrics Performances --- 
Specificity_RBF: 0.66
Sensitivity RBF: 0.96
RBF SVM accuracy:  0.84
RBF SVM precision:  0.83
RBF SVM recall:  0.96
RBF SVM F1-score:  0.89


In [20]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rfc = RandomForestClassifier(n_estimators=100, random_state=42)


rfc.fit(train_images.reshape(len(train_images), -1), train_labels)


pred_labels = rfc.predict(test_images.reshape(len(test_images), -1))

print(classification_report(test_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.91      0.87      0.89       198
           1       0.92      0.95      0.94       336

    accuracy                           0.92       534
   macro avg       0.92      0.91      0.91       534
weighted avg       0.92      0.92      0.92       534

